In [ ]:
#from collections import Counter
import pandas as pd
import numpy as np
#from imblearn.over_sampling import SMOTE
#from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
import os
import cv2
from PIL import Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
#%tensorflow_version  2.1.0
print(tf.__version__)
from keras.applications.mobilenet import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input,decode_predictions
from keras.models import Sequential

!pip install -q keras-tuner
import kerastuner as kt

2.3.0
     |████████████████████████████████| 71kB 7.9MB/s 


In [ ]:
inpf1 = np.load("/content/drive/MyDrive/drug_data/files/F1.npy")
inpf1.shape

(24324, 876)

In [ ]:
inpf1 = inpf1.reshape(24324,1,876)
inpf1.shape

(24324, 1, 876)

In [ ]:
inpf2 = np.load("/content/drive/MyDrive/drug_data/files/F2.npy")

In [ ]:
inpf2 = inpf2.reshape(24324,1,876)
inpf2.shape

(24324, 1, 876)

In [ ]:
altered = np.load("/content/drive/MyDrive/drug_data/files/altered_data.npy",allow_pickle=True)

In [ ]:
altered = altered.reshape(24324,1,876)
altered.shape

(24324, 1, 876)

In [ ]:
sqnp = np.load("/content/drive/MyDrive/drug_data/files/sqnp.npy")

In [ ]:
sqnp = sqnp.reshape(24324,1,876)
sqnp.shape

(24324, 1, 876)

In [ ]:
target = np.load("/content/drive/MyDrive/drug_data/files/altered_targets.npy",allow_pickle=True)
target.shape

(24324, 206)

In [ ]:
inpf1=tf.keras.utils.normalize(inpf1, axis=0, order=2)
inpf2=tf.keras.utils.normalize(inpf2, axis=0, order=2)
sqnp = tf.keras.utils.normalize(sqnp,axis=0,order=2)

In [ ]:
tf.keras.backend.set_floatx('float64')

In [ ]:
from functools import partial
reg_dense = partial(
    tf.keras.layers.Dense,
    activation = 'relu',
    kernel_initializer='he_normal',
    kernel_regularizer=keras.regularizers.l2(0.01)) 

In [ ]:
inputconv = tf.keras.Input(shape=(1,876))
inputconv1 = tf.keras.Input(shape=(1,876))
inputconv2 = tf.keras.Input(shape=(1,876))

x = tf.keras.layers.Reshape((12,73,1))(inputconv)
x1 = tf.keras.layers.Reshape((12,73,1))(inputconv1)
x2 = tf.keras.layers.Reshape((12,73,1))(inputconv2)


x = tf.keras.layers.Conv2D(100,2,
                           strides=(1,1),
                           padding='valid',
                           kernel_initializer='he_normal',
                           kernel_regularizer=tf.keras.regularizers.l1_l2(),
                           bias_constraint=tf.keras.constraints.non_neg())(x)
x = tf.keras.layers.Conv2D(64,2,
                           strides=(1,1),
                           padding='valid',
                           kernel_initializer='he_normal',
                           kernel_regularizer=tf.keras.regularizers.l1_l2(),
                           bias_constraint=tf.keras.constraints.non_neg())(x)                         
#x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
#x = reg_dense(500)(x)


#x1 = tf.keras.layers.Reshape((12,73,1))(inputconv1)
x1 = tf.keras.layers.Conv2D(100,2,
                           strides=(1,1),
                           padding='valid',
                           kernel_initializer='he_normal',
                           kernel_regularizer=tf.keras.regularizers.l1_l2(),
                           bias_constraint=tf.keras.constraints.non_neg())(x1)
x1 = tf.keras.layers.Conv2D(64,2,
                           strides=(1,1),
                           padding='valid',
                           kernel_initializer='he_normal',
                           kernel_regularizer=tf.keras.regularizers.l1_l2(),
                           bias_constraint=tf.keras.constraints.non_neg())(x1)
#x1 = tf.keras.layers.MaxPooling2D()(x1)
x1 = tf.keras.layers.GlobalAveragePooling2D()(x1)
#x1 = reg_dense(500)(x1)

x2 = tf.keras.layers.Conv2D(100,2,
                           strides=(1,1),
                           padding='valid',
                           kernel_initializer='he_normal',
                           kernel_regularizer=tf.keras.regularizers.l1_l2(),
                           bias_constraint=tf.keras.constraints.non_neg())(x2)
x2 = tf.keras.layers.Conv2D(64,2,
                           strides=(1,1),
                           padding='valid',
                           kernel_initializer='he_normal',
                           kernel_regularizer=tf.keras.regularizers.l1_l2(),
                           bias_constraint=tf.keras.constraints.non_neg())(x2)
#x2 = tf.keras.layers.MaxPooling2D()(x2)
x2 = tf.keras.layers.GlobalAveragePooling2D()(x2)


conv = tf.keras.layers.concatenate([x,x1])
conv = tf.keras.layers.Reshape((8,16,1))(conv)
conv = tf.keras.layers.Conv2D(64,(1,3),strides=1,
                                kernel_initializer=tf.keras.initializers.he_normal(),
                                kernel_regularizer=tf.keras.regularizers.l1_l2())(conv)
conv=tf.keras.layers.GlobalAveragePooling2D()(conv)

conv1 = tf.keras.layers.concatenate([x2,x1])
conv1 = tf.keras.layers.Reshape((16,8,1))(conv1)
conv1 = tf.keras.layers.Conv2D(64,(1,3),strides=1,
                                kernel_initializer=tf.keras.initializers.he_normal(),
                                kernel_regularizer=tf.keras.regularizers.l1_l2())(conv1)
conv1=tf.keras.layers.GlobalAveragePooling2D()(conv1)

conv2 = tf.keras.layers.concatenate([x,x2])
conv2 = tf.keras.layers.Reshape((32,4,1))(conv2)
conv2 = tf.keras.layers.Conv2D(64,(1,3),strides=1,
                                kernel_initializer=tf.keras.initializers.he_normal(),
                                kernel_regularizer=tf.keras.regularizers.l1_l2())(conv2)
conv2=tf.keras.layers.GlobalAveragePooling2D()(conv2)
#conv = tf.keras.layers.Conv2D(900,3,strides=(1,2),
 #                              kernel_initializer=tf.keras.initializers.he_normal(),
  #                              kernel_regularizer=tf.keras.regularizers.l1_l2())(conv)
#conv=tf.keras.layers.GlobalAveragePooling2D()(conv)
conc = tf.keras.layers.concatenate([conv,conv1,conv2])
g = reg_dense(1650)(conc)
g = reg_dense(1100)(g)
g = reg_dense(800)(g)
g = reg_dense(500)(g)
#g = reg_dense(290)(g)

output = tf.keras.layers.Dense(206,activation='softmax',name='main')(g)
#o1 = tf.keras.layers.Dense(206,activation='softmax',name='x')(x)
#o2 = tf.keras.layers.Dense(206,activation='softmax',name='x1')(x1)
#o3 = tf.keras.layers.Dense(206,activation='softmax',name='x2')(x2)
#o4 = tf.keras.layers.Dense(206,activation='softmax',name='convd')(conc)


model_conv = tf.keras.Model(inputs=[inputconv,inputconv1,inputconv2], outputs=[output])

In [ ]:
model_conv.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 1, 876)]     0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 1, 876)]     0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 1, 876)]     0                                            
__________________________________________________________________________________________________
reshape_12 (Reshape)            (None, 12, 73, 1)    0           input_7[0][0]                    
_______________________________________________________________________________________

In [ ]:
import keras.backend as K

p_min = 0.0001
p_max = 0.999
def logloss(y_true, y_pred):
    #y_pred = tf.clip_by_value(y_pred,p_min,p_max)
    return -K.mean(y_true*K.log(y_pred) + (1-y_true)*K.log(1-y_pred))

In [ ]:
model_conv.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.18,momentum=0.90),
    loss=[
          tf.keras.losses.MeanSquaredLogarithmicError(),
          #tf.keras.losses.MeanSquaredLogarithmicError(),
          #tf.keras.losses.MeanSquaredLogarithmicError(),
          #tf.keras.losses.MeanSquaredLogarithmicError(),
          #tf.keras.losses.MeanSquaredLogarithmicError(),
          #tf.keras.losses.MeanSquaredLogarithmicError()
          ],
    metrics = [
               #tf.keras.metrics.CategoricalCrossentropy(name='1cce'),
               #tf.keras.metrics.CategoricalCrossentropy(name='2cce'),
               #tf.keras.metrics.CategoricalCrossentropy(name='3cce'),
               logloss,
               tf.keras.metrics.Precision(),
               tf.keras.metrics.Recall(),
               tf.keras.metrics.CategoricalCrossentropy(label_smoothing=0.2)],
    #loss_weights=[0.10,0.8,0.03,0.03,0.03,0.05]
)

In [ ]:
history = model_conv.fit(x=[inpf1,inpf2,sqnp],y=target,epochs=40,batch_size=32)

Epoch 1/40
761/761 [==============================] - 64s 84ms/step - loss: 0.7461 - logloss: 0.0234 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00 - categorical_crossentropy: 4.1175
Epoch 2/40
194/761 [======>.......................] - ETA: 46s - loss: 0.7461 - logloss: 0.0233 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00 - categorical_crossentropy: 4.1065

KeyboardInterrupt: ignored